In [1]:
import pandas
import numpy as np
import sys
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
fs = open('path.txt', 'r')
path = fs.read()  
fs.close()
sys.path.append(path)

import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
import schedule

In [2]:
import numpy
import pandas as pd
def write_dict_h5(dfile, f, k, fbencodings, var_selection=[], mode='a', attrs={}): 
    """ Writes each separate variable from the observation or feedback tables inot netcdf using h5py.
          f is a pandas dataframe with one column, one for each variable
          k is either 'era5fb' or 'observations_table'
          fbencodings is the encodings of variable types, e.g. {'observations_id': { 'compression': 'gzip' } }
    """

    #attrs=  {'date_time':('units','seconds since 1900-01-01 00:00:00')}
    #attrs = {'observation_id': ('description', 'unique ID for observation'), 'report_id': ('description', 'Link to header information') , 'date_time':('units','seconds since 1900-01-01 00:00:00') }
    
    with h5py.File(dfile,mode) as fd:
        if k == '':
            v = var_selection[0]
            fd.create_dataset(v,f[v].shape,f[v].dtype,compression=fbencodings[v]['compression'], chunks=True)
            fd[v][:]=f[v].values[:]
        else:
            try:
                fd.create_group(k)
                index=numpy.zeros (f[list(f.keys())[0]].shape[0], dtype='S1')
                fd[k].create_dataset('index', data=index)
            except:
                pass
            if not var_selection:
                var_selection=list(f.keys())
            fixed_string_len = 20
            string10=numpy.zeros(fixed_string_len,dtype='S1')
            sdict={}
            slist=[]

            #groupencodings     

            for v in var_selection:          
                #variables_dic[v] = ''
                if type(f[v]) == pd.core.series.Series:
                    fvv=f[v].values
                else:
                    fvv=f[v]

                if type(fvv[0]) not in [str,bytes,numpy.bytes_]:

                    if fvv.dtype !='S1':

                        fd[k].create_dataset(v,fvv.shape,fvv.dtype,compression=fbencodings[v]['compression'], chunks=True)
                        fd[k][v][:]=fvv[:]
                        if attrs:    #  attrs={'date_time':('units','seconds since 1900-01-01 00:00:00')}
                            if v in attrs.keys():
                                for kk,vv in attrs[v].items():
                                    if type(vv) is str:  
                                        fd[k][v].attrs[kk]=numpy.bytes_(vv)
                                    else:
                                        fd[k][v].attrs[kk]=vv

                        if v in ['date_time','report_timestamp','record_timestamp']:
                            fd[k][v].attrs['units']=numpy.bytes_('seconds since 1900-01-01 00:00:00')                            #print (  fk, ' ' , v , ' ' ,   ) 

                    else:
                        fd[k].create_dataset(v,fvv.shape,fvv.dtype,compression=fbencodings[v]['compression'], chunks=True)
                        fd[k][v][:]=fvv[:]
                        slen=fvv.shape[1]
                        sdict[v]=slen
                        if slen not in slist:
                            slist.append(slen)
                            try:
                                fd[k].create_dataset( 'string{}'.format(slen),  data=string10[:slen]  )
                            except:
                                pass               
                        if v in attrs.keys():
                            fd[k][v].attrs['description']=numpy.bytes_(attrs[v]['description'])
                            fd[k][v].attrs['external_table']=numpy.bytes_(attrs[v]['external_table'])

                else:
                    sleno=len(fvv[0])
                    slen=sleno
                    try:
                        slen=int(fvv.dtype.descr[0][1].split('S')[1])
                    except:  
                        pass

                    sdict[v]=slen
                    if slen not in slist:
                        slist.append(slen)
                        try:
                            print('trying to cread dim')
                            fd[k].create_dataset( 'string{}'.format(slen),  data=string10[:slen]  )
                            print('dim created')
                        except:
                            print('dim creation failed')
                            pass               

                    #x=x.reshape(fvv.shape[0],slen)
                    fd[k].create_dataset(v,data=fvv.view('S1').reshape(fvv.shape[0],slen),compression=fbencodings[v]['compression'],chunks=True)
                    if v in attrs.keys():
                        fd[k][v].attrs['description']     =numpy.bytes_(attrs[v]['description'])
                        fd[k][v].attrs['external_table']=numpy.bytes_(attrs[v]['external_table'])                

                #variables_dic[v] = f[v].values.dtype

            for v in fd[k].keys(): #var_selection:
                l=0      

                '''
                if v == 'primary_station_id':
                    try:
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                    except:
                        pass

                    try:
                        slen =  len( fd[k][v][0] )
                        stringa=numpy.zeros( slen , dtype='S1')
                        fd[k].create_dataset( 'string{}'.format(slen),  data= stringa  )                                        
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                                        
                    except:
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                    


                if v == 'station_name':
                    try:
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                        slen =  len( fd[k][v][0][0])
                        stringa=numpy.zeros( slen , dtype='S1')
                    except:
                        pass
                    try:
                        fd[k].create_dataset( 'string{}'.format(slen),  data= stringa  )
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                    
                        print('done attaching')
                    except:
                        print('not working')

                '''             
                try:
                    if type(f[v]) == pd.core.series.Series:
                        fvv=f[v].values
                    else:
                        fvv=f[v]
                    if 'string' not in v and v!='index':                    
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                        #print(v,fvv.ndim,type(fvv[0]))
                        if fvv.ndim==2 or type(fvv[0]) in [str,bytes,numpy.bytes_]:
                            slen=sdict[v]
                            #slen=10
                            fd[k][v].dims[1].attach_scale(fd[k]['string{}'.format(slen)])
                except:
                    pass



            i=4        
            for v in slist:
                s='string{}'.format(v)
                for a in ['NAME']:
                    fd[k][s].attrs[a]=numpy.bytes_('This is a netCDF dimension but not a netCDF variable.')

                i+=1

    return
#variables_dic = {}

In [3]:
# files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035*.nc')
files = glob.glob('/raid60/scratch/uli/0-20000-0-96*.nc')
files[0]

'/raid60/scratch/uli/0-20000-0-96035_CEUAS_merged_v0.nc'

In [4]:
data = eua.CDMDataset(files[0])
data

File: <HDF5 file "0-20000-0-96035_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/0-20000-0-96035_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (17717,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 55
 - G | observations_table___________________________ : : 50
 - G | observed_variable____________________________ : : 9
 - V | recordindex__________________________________ : : (48757,)
 - V | recordtimestamp______________________________ : : (48757,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 47
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ :

In [5]:
allvars = data.observations_table.observed_variable[()]
np.unique(allvars)

array([  0,  34,  36,  38,  39,  85, 104, 105, 106, 107, 117, 136, 137,
       138, 139, 140])

In [6]:
allvars = data.observations_table.observed_variable[()]
allvars.sort()
allvars = np.unique(allvars)
#
ri = data.recordindex[()]
print('recordindex: ', len(ri))
# rt = data.recordtimestamp[()]
keys = data.observations_table.keys()[:-1]
# dropping all keys, where dimensions won't work - just help variabels for dimensions
keys.pop(43)
keys.pop(42)
keys.pop(41)
recordindices = [[] for i in range(len(allvars))]
recordtimestamps = [[] for i in range(len(allvars))]

# output variables (from observations_table)
ov = []
for o in keys:
    ov.append([[] for i in range(len(np.unique(allvars)))])

#
# load data into memory and decode byte arrays
#
print('loading data')
obsv = data.observations_table.observed_variable[:]
ov_vars = []
for o in range(len(keys)):
    ov_vars.append(data.observations_table[keys[o]][:])

#
# resorting the data
#
print('resort:start')
for i in range(len(ri)):
    try:
        start = ri[i]
        end = ri[i+1]
    except:
        start = ri[i]
        end = len(data.observations_table.observed_variable[()])
        
    a = obsv[start:end]
    sa, sortindex = zip(*sorted(zip(a, range(len(a)))))
    sortindex = np.array(sortindex)
    helpvar = []
    for o in range(len(keys)):
        b = ov_vars[o][start:end]
        helpvar.append(b[sortindex])

    for j in range(len(allvars)):
        for k in range(len(sa)):
            if sa[k] == allvars[j]:
                for m in range(len(helpvar)):
                    ov[m][j].append(helpvar[m][k])
        recordindices[j].append(len(ov[0][j]))

print('resort:done')
        
#
# setting record_indices to the right value -> stacking them
#
lenadd = 0.
for i in range(len(recordindices)):
    recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
    lenadd += len(ov[0][i])

#
# shaping record_indices -> setting every missing value to nan
#
print('shaping record_indices')
old = -1
for i in range(len(recordindices)):
    for j in range(len(recordindices[i])):
        if i != 0 or j != 0:
            if recordindices[i][j] == recordindices[i][j-1]:
                old = recordindices[i][j]
                recordindices[i][j] = np.nan
            elif recordindices[i][j] == old:
                recordindices[i][j] = np.nan

#
# recordtimestamps are only necessary once
#
recordtimestamps = recordtimestamps[0]

# 
# stacking all output variables
#
print('stacking output variables')
out = []
for k in ov:
    out.append([j for i in k for j in i])

recordindex:  48757
loading data
resort:start
resort:done
shaping record_indices
stacking output variables


In [8]:
# targetfile has to be a copy of the original file
targetfile = '/raid60/scratch/uli/testdata/0-20000-0-96035_CEUAS_merged_v0.nc'
file = h5py.File(targetfile, 'r+')

#
# writing data into observations_table
#
for i in range(len(keys)):
    try:
        del file['observations_table'][keys[i]]
    except:
        pass
    if keys[i] == 'index':
        pass
    elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
        slen = len(out[i][0])
        alldict = {keys[i]:np.asarray(out[i], dtype='S{}'.format(slen))}
        write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
    else:
        alldict = pandas.DataFrame({keys[i]:out[i]})
        write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  
#
# writing the recordindices and recordtimestamp.
#       
for i in range(len(recordindices)):
    try:
        del file['recordindices'][str(allvars[i])]
    except:
        pass
    testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
    write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': 'gzip' } }, [str(allvars[i])]) 

try:
    del file['recordindex']
except:
    pass


In [9]:
targetfile = '/raid60/scratch/uli/testdata/0-20000-0-96035_CEUAS_merged_v0.nc'
data_test = eua.CDMDataset(targetfile)
data_test

File: <HDF5 file "0-20000-0-96035_CEUAS_merged_v0.nc" (mode r+)>
Filename: /raid60/scratch/uli/testdata/0-20000-0-96035_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (17717,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 55
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 17
 - V | recordtimestamp______________________________ : : (48757,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 47
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units______________________________________

In [10]:
data_test.observations_table

observations_table:

adjustment_id_____________________________________ : : (7289056,)
advanced_assimilation_feedback____________________ : : (7289056,)
advanced_homogenisation___________________________ : : (7289056,)
advanced_qc_______________________________________ : : (7289056,)
advanced_uncertainty______________________________ : : (7289056,)
bbox_max_latitude_________________________________ : : (7289056,)
bbox_max_longitude________________________________ : : (7289056,)
bbox_min_latitude_________________________________ : : (7289056,)
bbox_min_longitude________________________________ : : (7289056,)
code_table________________________________________ : : (7289056,)
conversion_flag___________________________________ : : (7289056,)
conversion_method_________________________________ : : (7289056,)
crs_______________________________________________ : : (7289056,)
data_policy_licence_______________________________ : : (7289056,)
date_time_________________________________________ : : 

In [11]:
data_test.recordindices

recordindices:

0_________________________________________________ : : (48757,)
104_______________________________________________ : : (48757,)
105_______________________________________________ : : (48757,)
106_______________________________________________ : : (48757,)
107_______________________________________________ : : (48757,)
117_______________________________________________ : : (48757,)
136_______________________________________________ : : (48757,)
137_______________________________________________ : : (48757,)
138_______________________________________________ : : (48757,)
139_______________________________________________ : : (48757,)
140_______________________________________________ : : (48757,)
34________________________________________________ : : (48757,)
36________________________________________________ : : (48757,)
38________________________________________________ : : (48757,)
39________________________________________________ : : (48757,)
85______________________

In [12]:
data_test.observations_table.observed_variable[0:-1:2000]

array([  0,  34,  34, ..., 140, 140, 140])

In [13]:
keys[-3]

'z_coordinate'

In [14]:
data_test.recordindices['85'][:50]

array([942446.,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan])

In [15]:
data_test.observations_table.z_coordinate[:36]

array([   245.16624451,  34500.        ,  40000.        ,  43100.        ,
        50000.        ,  56700.        ,  70000.        ,  73800.        ,
        77100.        ,  85000.        , 100000.        , 100700.        ,
        25400.        ,  30000.        ,  40000.        ,  50000.        ,
        67700.        ,  70000.        ,  74500.        ,  85000.        ,
        94000.        , 100000.        , 100600.        ,  25100.        ,
        30000.        ,  40000.        ,  41900.        ,  50000.        ,
        51300.        ,  61300.        ,  70000.        ,  75000.        ,
        80100.        ,  85000.        ,  97800.        , 100000.        ])

---

In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# # recordindices einzeln sortieren - für alle variablen eigene listen erstellen - dazu recordindices erstellen - alle daten hintereinander legen - len aller vorhergehender rec_in zu den nachkommenden addieren.
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# variables = [[] for i in range(len(np.unique(allvars)))]
# values = [[] for i in range(len(np.unique(allvars)))]
# zcoords = [[] for i in range(len(np.unique(allvars)))]

# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     b = data.observations_table.observation_value[()][start:end]
#     c = data.observations_table.z_coordinate[()][start:end]

#     sa, sb = zip(*sorted(zip(a, b)))
#     sa, sc = zip(*sorted(zip(a, c)))
#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 variables[j].append(sa[k])
#                 values[j].append(sb[k])
#                 zcoords[j].append(sc[k])
#         recordindices[j].append(len(variables[j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])
    
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = data.observations_table[keys[o]][()][start:end]
#         try: # all varibales with different shape won't work here
#             sa, sb = zip(*sorted(zip(a, b)))
#             helpvar.append(sb)
#         except:
#             pass

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     try: # all varibales with different shape won't work here
#                         ov[m][j].append(helpvar[m][k])
#                     except:
#                         pass
#         recordindices[j].append(len(ov[0][j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# #
# #
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print(len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])    
    
# print('resort:start')
# for i in range(len(ri)):
#     print(i)
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#             b = []
#             for n in data.observations_table[keys[o]][()][start:end]:
#                 c = ''
#                 for bb in n:
#                     c = c + bb.decode()
#                 b.append(c)
#         else:
#             b = data.observations_table[keys[o]][()][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# #
# for o in range(len(keys)):
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in range(len(out[o])):
#             c = []
#             for bb in out[o][n]:
#                 c.append(bb.encode())
#             out[o][n] = c


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print('recordindex: ', len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])

# #
# # load data into memory and decode byte arrays
# #
# print('loading data')
# obsv = data.observations_table.observed_variable[:]
# ov_vars = []
# for o in range(len(keys)):
#     ov_vars.append(data.observations_table[keys[o]][:])
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in ov_vars[o]:
#             c = ''
#             for bb in n:
#                 c = c + bb.decode()
#             b.append(c)
#         ov_vars[o] = b

# #
# # resorting the data
# #
# print('resort:start')
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = obsv[start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = ov_vars[o][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# print('shaping record_indices')
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# print('stacking output variables')
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# # this takes very long -> find faster option
# #
# print('restoring byte arrays')
# bytelist = [25, 34, 38, 39]
# for o in bytelist:
#     originallen = len(data.observations_table[keys[o]][()][0])
#     for n in range(len(out[o])):
#         c = [elem.encode() for elem in out[o][n]]
#         # problem: if the string was [b'x', b'y', b''] in the first place, it get converted to 'xy' and then back to [b'x', b'y']
#         # add empty byte strings until the data is as long as before:
#         while(len(c) < originallen):
#             c.append(str('').encode())
#         out[o][n] = c